# **1) Introducción**

El problema al que nos enfrentamos es programar un robot para que, dado un mapa (que simula un almacén), coja las diferentes mesas (inventarios) y las lleve a su puesto de recogida por parte de los empleados de esa empresa. Este robot, deberá buscar el camino más corto posible, de forma rápida y evitando chocar con las paredes del almacén.

Para desarrollar el código, hemos hecho la clase *Nodo* para transformar las diferentes posiciones del mapa en un grafo, y la función *AEstrella*, para que el robot pueda moverse por el mapa de la forma más óptima posible dada la heurística a utilizar en el enunciado, calculando qué camino tomar en cada caso en base al grafo formado. Además hemos realizado otras funciones de lectura y visualización del mapa, de forma que se pueda seguir de manera visual la secuencia de acciones tomada por el robot.

Por último tenemos la implementación principal del código, en la cual pasamos el mapa al robot, con los distintos puntos de recogida y entrega. Esta implementación principal es la que, mediante búsqueda heurística, va elaborando poco a poco el plan de actuación del robot, eligiendo el camino más corto a los diferentes puntos, en cada caso.

# **2) Desarrollo de la ejecución**

## **2.1) Asunciones**

Para el desarrollo del código hemos realizado las siguientes asunciones:

1. El robot se ha contemplado similar al del vídeo del enunciado del ejercicio, por tanto, se considerará que éste se deberá situar en la misma casilla que el inventario y su zona de depósito a la hora de recogerlo/dejarlo.
2. Se considera que, cuando el robot deja el inventario en el depósito, éste es manipulado y retirado automáticamente por un operador. Por tanto, ese espacio en el mapa se considerará como "." o cero para futuros movimientos del robot, indicando que continuará siendo transitable por el robot, pero que ya no corresponderá a ninguna posición objetivo donde se deba de mover el robot (de esta forma es más realista).

3. Se estima que todos los puntos objetivo (tanto para recogida como para depósito) son alcanzables por el robot y que no aparecerán obstáculos o zonas consideradas como "pared" que no estuvieran en un principio en el mapa, por lo que el plan siempre tendrá solución.
4. Se contempla que el robot conoce las posiciones de los objetos y el punto de recogida del paquete que está portando antes de comenzar a elaborar el plan. Las listas de posiciones de recogida y dejada de piezas, por tanto, se inicializarán con los valores correspondientes al principio del programa, basándonos en la teoría que el encargado de Amazon correspondiente sería quien los hubiera introducido vía software propio del controlador del robot.
5. Se estima que el robot funcionará por búsqueda de objetos de manera reactiva, por lo que su plan de acción se irá construyendo de forma rápida y objetivo a objetivo, calculando qué inventario se encuentra más cercano y empleando, para ello, la heurística de Manhattan.
6. Se considera que el robot puede pasar por encima de las casillas que contienen los inventarios, aunque éste tenga cargado otro inventario. Además, las posiciones marcadas en el mapa como zona objetivo o zona de recogida también serán transitables por el robot. Esta asunción no sería realista ya que, en la realidad, no sería lógico que pudiera pasar el robot cargado por encima de otro inventario, pero no se ha contemplado para el ejercicio porque, en nuestro caso, siempre cogerá primero M2 antes que M1 ya que la heurística para recoger el primero será menor a la del segundo.

## **2.2) Código y explicación**

Primero generamos un archivo .txt del mapa de la actividad en el mismo path donde se encuentra el archivo. De esta manera, se consigue un código que genera el mapa (en el mismo directorio, un archivo .txt) que se usa para la resolución del problema.

In [ ]:
import os
mapaActividad2 = os.getcwd() # este comando busca el path donde se encuentra el archivo.

file = open ("mapaActividad2.txt", "w") # w escritura , r solo lectura. Usamos escritura porque vamos a sobreescribirlo. 

file.write("######" +os.linesep)
file.write("#A#.C#" +os.linesep)
file.write("#.#..#" +os.linesep)
file.write("#B.R.#" +os.linesep)
file.write("#.ZYX#" +os.linesep)
file.write("######" +os.linesep)
file.close()

Comenzamos definiendo la clase Nodo (Roy, 2019) para generar objetos sobre los que estructurar el grafo en el programa, y definiendo las funciones de distancia entre dos nodos (Distancia Manhattan), que será la heurística del problema, y de buscar posición, para saber la posición inicial del robot en el mapa.

In [ ]:
class Nodo:
    def __init__(self, posicion=[0, 0], anterior=None):
        self.posicion = posicion
        self.anterior = anterior
        #h: Estimación del coste adicional necesario para alcanzar el nodo objetivo desde el nodo actual
        self.h = distancia(self.posicion, pos_objetivo)
 
        if self.anterior == None:
            self.g = 0
        else:
          #g: Medida del coste para ir desde el estado inicial hasta el nodo actual
            self.g = self.anterior.g + 1
          #f: Suma de g y h
        self.f = self.g + self.h


# Distancia Manhattan: Suma de las diferencias absolutas de las coordenadas entre dos puntos.
def distancia(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1]) 

# Función para encontrar un valor dado en el mapa.
def buscarPos(x, mapa):
    for i in range(len(mapa[0])):
        for j in range(len(mapa[1])):
            if mapa[i][j] == x:
                return [i, j]
    return 0

A continuación definimos una serie de funciones que nos van a servir para ejecutar nuestro algoritmo de AEstrella (A*) (Guerra, 2010). 

1. vecinos: busca los nodos vecinos a los que se puede acceder
2. f_menor: elimina el elemento con menor f de la lista abierta y lo pasa a la lista cerrada
3. en_lista: comprueba si un nodo está en una lista
4. ruta: gestiona los nodos vecinos
5. aobjetivo: comprueba si el objetivo está en la lista abierta
6. camino: devuelve una lista con las posiciones del camino a seguir

In [ ]:
# Devuelve una lista con los nodos vecinos a los que podemos acceder.
def vecinos(nodo, mapa):
    vecinos = []
    x=nodo.posicion[0]
    y=nodo.posicion[1]
    #Si se puede mover a la derecha
    if mapa[x+1][y] != 1:
        vecinos.append(Nodo([x+1,y], nodo))
    #Si se puede mover a la izquierda
    if mapa[x-1][y] != 1:
        vecinos.append(Nodo([x-1, y], nodo))
    #Si se puede mover hacia abajo
    if mapa[x][y-1] != 1:
        vecinos.append(Nodo([x, y-1], nodo))
    #Si se puede mover hacia arriba
    if mapa[x][y+1] != 1:
        vecinos.append(Nodo([x, y+1], nodo))
    return vecinos

# Elimina el elemento de la lista abierta con menor f y lo pasa a la lista cerrada
def f_menor(l_abierta,l_cerrada):
    a = l_abierta[0]
    n = 0
    for i in range(1, len(l_abierta)):
        if l_abierta[i].f < a.f:
            a = l_abierta[i]
            n = i
    l_cerrada.append(l_abierta[n])
    del l_abierta[n]

# Comprueba si un nodo está en una lista.
def en_lista(nodo, lista):
    for i in range(len(lista)):
        if nodo.posicion == lista[i].posicion:
            return 1
    return 0

# Gestiona los vecinos del nodo seleccionado.
def ruta(nodos, l_abierta, l_cerrada, select):
    for i in range(len(nodos)):
        nodo = nodos[i]
        if en_lista(nodo, l_cerrada):
            continue
        elif not en_lista(nodo, l_abierta):
            l_abierta.append(nodo)
        else:
            if select.g+1 < nodo.g:
                for j in range(len(l_abierta)):
                    if nodo.posicion == l_abierta[j].posicion:
                        del l_abierta[j]
                        l_abierta.append(nodo)
                        break

# Comprueba si el objetivo objetivo está en la lista abierta.
def aobjetivo(l_abierta, nodofin):
    for i in range(len(l_abierta)):
        if nodofin.posicion == l_abierta[i].posicion:
            return 0
    return 1

# Retorna una lista con las posiciones del camino a seguir.
def camino(l_abierta, nodofin):
    for i in range(len(l_abierta)):
        if nodofin.posicion == l_abierta[i].posicion:
            objetivo = l_abierta[i]

    camino = []
    while objetivo.anterior != None:
        camino.append(objetivo.posicion)
        objetivo = objetivo.anterior
    camino.reverse()
    return camino



Una vez definidas estas funciones, se genera la función AEstrella, donde implementaremos la heurística del problema para que el robot decida que camino tomar. Este algoritmo se basa en la búsqueda general. Se debe añadir el valor g a cada nodo expandido y posteriormente ordenar la lista abierta en base a los valores crecientes de f. Se añadirán, entonces, los nuevos nodos en la lista abierta según sus valores crecientes de f. 


In [ ]:

    def AEstrella (mapa,inicio,fin)# Nodos de inicio y fin.
    ninicio = Nodo(inicio)
    nfin = Nodo(fin)
    
    # Definimos las listas ABIERTA y CERRADA:
    #Lista abierta: Nodos que se generan y a los que se les aplica la función heurística. Aún no han sido examinados.
    lista_abierta = []
    #Lista cerrada: Nodos que ya se han examinado. 
    #Se necesita para comprobar si un nuevo nodo ya ha sido generado anteriormente.
    lista_cerrada = []
    
    # Añadir vecinos a la lista abierta
    lista_abierta += vecinos(ninicio, mapa)
    
    # Añadir nodo inicial a la lista cerrada.
    lista_cerrada.append(ninicio)
                             
    # El algoritmo continúa mientras que no estemos en la lista cerrada.
    while aobjetivo(lista_abierta,nfin):
      
      # Analiza el último elemento de la lista cerrada.
        f_menor(lista_abierta, lista_cerrada)
        select = lista_cerrada[-1]
        nodos = vecinos(select,mapa)
        ruta(nodos,lista_abierta,lista_cerrada,select
    return camino(lista_abierta,nfin)

Para dejar un registro en pantalla de los pasos que ha seguido el robot, se definen algunas funciones de interés a la hora de visualizar el mapa:

In [ ]:
# Quita el ultimo carácter de una lista. Lo necesitamos para poder mostrar el mapa de forma correcta.
def quitarUltimo(lista):
    for i in range(len(lista)):
        lista[i] = lista[i][:-1]
    return lista

# Función para mostrar el mapa
def visualizarMapa(mapa):
    for i in range(len(mapa)):
        print(mapa[i])

# Función para leer el mapa de tal forma que se pueda operar con él.
def leerMapa(mapaRAW):
    mapa= []
    for r in range(len(mapaRAW[0])):
        mapa.append([])
          for c in range(len(mapaRAW[1])):
            #print(c)
            if mapaRAW[r][c] == '.':
                mapa[r].append(0)
            if mapaRAW[r][c] == "#":
                mapa[r].append(1)
            if mapaRAW[r][c] == "R":
                mapa[r].append(2)
            if mapaRAW[r][c] == "A" or mapaRAW[r][c] == "B" or mapaRAW[r][c] == "C":
                mapa[r].append(3)
            if mapaRAW[r][c] == "X" or mapaRAW[r][c] == "Y" or mapaRAW[r][c] == "Z":
                mapa[r].append(4)
        return mapa

# Función para sumar 1 a los elementos del mapa. 
# Esto se hace porque vamos a considerar las paredes exteriores y por tanto añadimos una fila y una columna.
def convertirObjetivos(objetivos,depositos):
    for a in range(len(objetivos)):
        for b in range(len(objetivos[a])):
            objetivos[a][b] = objetivos[a][b] + 1
            depositos[a][b] = depositos[a][b] + 1
    return [objetivos,depositos]

Creamos dos funciones para que impriman los pasos que el robot va a estar realizando.
1. print_Coger: para imprimir el camino de ida hacia la mesa y la acción de coger la mesa
2. print_Dejar: imprime el camino de ida al depósito y la acción de dejar la mesa.




In [ ]:
# Muestra los pasos seguidos hasta llegar y coger el objetivo.
def print_Coger(busqueda_camino, pos_objetivo):
    print("\n")
    for i in range(len(busqueda_camino)):
        print("-Mover Robot a la fila '{}' y columna '{}'".format(busqueda_camino[i][0], busqueda_camino[i][1]))
        if busqueda_camino[i] == pos_objetivo:          
            print("-Coger '{}' en la fila '{}' y columna '{}'".format(nombres[objetivo],busqueda_camino[i][0], 
                                                                      busqueda_camino[i][1]))
        
# Muestra los pasos seguidos hasta llegar y dejar el objetivo.
def print_Dejar(busqueda_camino, pos_objetivo):
    for i in range(len(busqueda_camino)):
        print("-Mover Robot a la fila '{}' y columna '{}'".format(busqueda_camino[i][0], busqueda_camino[i][1]))
        if busqueda_camino[i] == pos_objetivo:
            print("-Dejar '{}' en la fila '{}' y columna '{}'".format(nombres[objetivo],busqueda_camino[i][0], 
                                                                      busqueda_camino[i][1])) 

Finalmente, pasamos la lista de objetivos y sus respectivos depósitos al robot para que este decida el órden de las recogidas y las lleve al lugar de recogida.

A continuación, se especifican las posiciones de los tres inventarios M1, M2 y M3, tanto en su posición inicial en el mapa del enunciado, como en la posición final en la que se deben ubicar. Para recoger y asociar de forma sencilla las posiciones a cada inventario, se crearán tres listas con toda la información requerida por cada uno de estos, de forma que el primer valor de cada lista corresponderá al inventario 1, la segunda al inventario 2 y así sucesivamente.

In [ ]:
nombres =["M1","M2","M3"]
objetivos=[[0,0],[2,0],[0,3]]
depositos=[[3,3],[3,2],[3,1]]
cajasAlmacenadas=0

Se muestran los objetivos a dónde quiere llegar nuestro robot a recoger los paquetes, y los depositos donde debe dejar dichos paquetes:

In [ ]:
objetivos,depositos = convertirObjetivos(objetivos, depositos)
print("Los objetivos se encuentran en las posiciones: '{}' y los depósitos en las posiciones '{}'".format(objetivos,
                                                                                                          depositos))


Los objetivos se encuentran en las posiciones: '[[1, 1], [3, 1], [1, 4]]' y los depósitos en las posiciones '[[4, 4], [4, 3], [4, 2]]'


Se visualiza el mapa que tenemos y se traduce en un formato con el que podamos operar.

In [ ]:
mapaRaw = open('mapaActividad2.txt', "r")
mapaRaw = mapaRaw.readlines()
mapaRaw = quitarUltimo(mapaRaw)


print("* Mapa de la actividad\n")
visualizarMapa(mapaRaw)
mapaRaw= leerMapa(mapaRaw)

print("\n* Mapa con el que va a trabajar nuestro algoritmo: \n")
visualizarMapa(mapaRaw)

* Mapa de la actividad

######
#A#.C#
#.#..#
#B.R.#
#.ZYX#
######

* Mapa con el que va a trabajar nuestro algoritmo: 

[1, 1, 1, 1, 1, 1]
[1, 3, 1, 0, 3, 1]
[1, 0, 1, 0, 0, 1]
[1, 3, 0, 2, 0, 1]
[1, 0, 4, 4, 4, 1]
[1, 1, 1, 1, 1, 1]


Lo que tenemos en este mapa es:
* 0 - Posiciones en las que el robot se puede mover libremente.
* 1 - Posiciones a las cuales el robot no puede llegar (paredes).
* 2 - Posición del robot
* 3 - Posiciones a las que tiene que ir el robot para recoger los paquetes.
* 4-  Posiciones a las que tiene que ir el robot a dejar dichos paquetes.

Finalmente, se muestran los pasos que ha seguido el robot. Se ha de tener en cuenta que contamos con que existe la fila 0 y la columna 0.

In [ ]:
print("\nPasos a seguir para la resolución del problema: \n")
visualizarMapa(mapaRaw)
robot=buscarPos(2,mapaRaw)
print("-Robot inicialmente situado en fila '{}' y columna '{}'".format(robot[0],robot[1]))
mapaRaw[robot[0]][robot[1]]= 0
numeroObjetivos= len(objetivos)
pos_anterior=[]
while (cajasAlmacenadas < numeroObjetivos):
    costeAnterior=99999
    objetivo=0
    pos_objetivo=[]
    #Seleccionamos el menor coste
    for x in range(len(objetivos)):
        coste= distancia(robot,objetivos[x])
        if coste<costeAnterior:
            costeAnterior=coste
            objetivo = x
    pos_objetivo=objetivos[objetivo]
    # Busca el menor camino hacia cualquier paquete
    busqueda = AEstrella(mapaRaw, robot, pos_objetivo)
    print_Coger(busqueda,pos_objetivo)
    if len(pos_anterior)>0:
        mapaRaw[pos_anterior[0]][pos_anterior[1]]= 0
    mapaRaw[pos_objetivo[0]][pos_objetivo[1]]= 2
    pos_anterior=pos_objetivo
    visualizarMapa(mapaRaw)
    print("\n")

    # Cambiamos la posición del robot.
    robot = objetivos[objetivo]    
    
    # Busca el menor camino hacia el lugar donde debe dejar el paquete correspondiente.
    busqueda = AEstrella(mapaRaw, robot, depositos[objetivo])
    print_Dejar(busqueda,depositos[objetivo])  
    mapaRaw[pos_anterior[0]][pos_anterior[1]]= 0 
    mapaRaw[depositos[objetivo][0]][depositos[objetivo][1]]=2
    pos_anterior=depositos[objetivo]
    visualizarMapa(mapaRaw)
    robot=depositos[objetivo] 

    # Eliminamos tanto objetivos como depósitos.
    objetivos.remove(objetivos[objetivo])
    nombres.remove(nombres[objetivo])
    depositos.remove(depositos[objetivo])
    cajasAlmacenadas+=1

    

print("\n\nColocadas todas las cajas en su lugar correspondiente.\nFin del proceso.")


Pasos a seguir para la resolución del problema: 

[1, 1, 1, 1, 1, 1]
[1, 3, 1, 0, 3, 1]
[1, 0, 1, 0, 0, 1]
[1, 3, 0, 2, 0, 1]
[1, 0, 4, 4, 4, 1]
[1, 1, 1, 1, 1, 1]
-Robot inicialmente situado en fila '3' y columna '3'


-Mover Robot a la fila '3' y columna '2'
-Mover Robot a la fila '3' y columna '1'
-Coger 'M2' en la fila '3' y columna '1'
[1, 1, 1, 1, 1, 1]
[1, 3, 1, 0, 3, 1]
[1, 0, 1, 0, 0, 1]
[1, 2, 0, 0, 0, 1]
[1, 0, 4, 4, 4, 1]
[1, 1, 1, 1, 1, 1]


-Mover Robot a la fila '4' y columna '1'
-Mover Robot a la fila '4' y columna '2'
-Mover Robot a la fila '4' y columna '3'
-Dejar 'M2' en la fila '4' y columna '3'
[1, 1, 1, 1, 1, 1]
[1, 3, 1, 0, 3, 1]
[1, 0, 1, 0, 0, 1]
[1, 0, 0, 0, 0, 1]
[1, 0, 4, 2, 4, 1]
[1, 1, 1, 1, 1, 1]


-Mover Robot a la fila '3' y columna '3'
-Mover Robot a la fila '2' y columna '3'
-Mover Robot a la fila '1' y columna '3'
-Mover Robot a la fila '1' y columna '4'
-Coger 'M3' en la fila '1' y columna '4'
[1, 1, 1, 1, 1, 1]
[1, 3, 1, 0, 2, 1]
[1, 0, 1, 0, 0, 1

# **3) Dificultades encontradas**

* Llevó mucho tiempo intentar leer el archivo para analizar las casillas una a una ya que daba errores por la inicialización de las matrices. Finalmente, se optó por crear una lista y cada vez que se leía una fila del archivo .txt se agregaba a dicha lista. Además, en la lectura del .txt se tenía que quitar el ultimo carácter correspondiente al \n de salto de línea. Para solucionar esto se creó la función: quitarUltimo.
*	A la hora de grabar los movimientos que hacía el robot, tanto al recoger los objetivos, como para dejarlos en su posición, se buscó actualizar la posición del robot en el mapa para registrar dónde está el robot en ese paso (el 2 en dicha posición). En un principio, se quedaban grabados en el mapa múltiples 2 en posiciones en las que había pasado el robot, cuando debían de ser 0, pues el robot ya no se encontraba en dicha posición. Para que la representación del mapa de forma gráfica fuera correcta, se tuvo que generar una variable auxiliar (*pos_anterior*) para poder solventarlo.
*	Al añadir las paredes exteriores, la matriz cambia de ser 4x4 a 6x6. Por ello, todas las posiciones cambian y se debe definir una función para arreglar este problema.








# **4) Conclusiones**

* El robot ha conseguido llevar los tres inventarios a su destino en el mapa.

* El robot no se sale del mapa ni tampoco atraviesa las paredes definidas en éste, y además, para este ejericio en concreto, genera la mejor actuación elaborando un plan óptimo y encontrando la solución del problema. Se puede concluir, entonces, que el problema se ha resuelto satisfactoriamente.

* Manualmente, se calculó que para que se diese el número menor de pasos posibles, no importa si el robot empieza cogiendo M2 o M3, ya que en ambos da 24 pasos. Nuestro programa, al basarse en la distancia de Manhattan, ejecuta M2-M3-M1 en lugar de M3-M2-M1. En ambos casos, aún así, los pasos totales que se dan son 24.
Manualmente, se puede calcular que si el robot pudiese desplazarse sin tener en cuenta los almacenes, el proceso más óptimo sería M1-M3-M2, que daría 23 pasos. Pero como M1 es el almacén más alejado en cualquier momento para la distancia Manhattan, es el último que se recoge. 
Si se prueban a eliminar las paredes de las posiciones (2,1) y (2,2), el algoritmo también hace los pasos M2-M3-M1. ¿Sería más óptimo si, en este caso, hiciese el orden de M1-M3-M2? Es cierto que da más pasos al principio a la hora de llegar a M1, pero después, al ir a M3 y a M2, daría menos pasos y sería, en términos globales, más eficiente.
En cualquier caso, se prima la rapidez de obtención del plan a su optimización, por lo que la decisión se realiza en base a la menor distancia de Manhattan entre el robot y la posición a la que se desea ir, y esto, en ciertas ocasiones, podrá llevar a la resolución del problema de manera menos óptima que otro proceso de planificación más deliberativo.

# **5) Referencias bibliográficas**

- Guerra, A. (2010). Pathfinding A* en Python. Parte II. Razón Artificial. http://razonartificial.com/2010/04/pathfinding-a-en-python-parte-ii/

- Roy, B. (2019). A-Star (A*) Search Algorithm. Towards Data Science. https://towardsdatascience.com/a-star-a-search-algorithm-eb495fb156bb


